In [1]:
tickers = ['PETR4.SA' , 'BBAS3.SA'] #definimos os tickers das ações
periodo = '5y'



In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go 
import yfinance as yf

def fix_columns(df):
    return ['IBOV' if col == '^BVSP' else col.rstrip('.SA') for col in df.columns]


prices = yf.download(tickers=tickers , period=periodo, rounding  = True)['Adj Close']
prices.columns = fix_columns(prices)
prices.dropna(inplace=True)
retorno = prices.pct_change().dropna()
ativos = retorno.columns.to_list()

retorno


[*********************100%%**********************]  2 of 2 completed


,BBAS3,PETR4
Date,,
2019-06-14,-0.018133,-0.004251
2019-06-17,-0.000543,0.001067
2019-06-18,0.023370,0.012793
2019-06-19,0.016463,0.003158
2019-06-21,0.007315,0.027282
...,...,...
2024-06-07,-0.016918,-0.037513
2024-06-10,0.001122,0.015033
2024-06-11,0.010837,0.004388


In [3]:
prices

retorno

,BBAS3,PETR4
Date,,
2019-06-14,-0.018133,-0.004251
2019-06-17,-0.000543,0.001067
2019-06-18,0.023370,0.012793
2019-06-19,0.016463,0.003158
2019-06-21,0.007315,0.027282
...,...,...
2024-06-07,-0.016918,-0.037513
2024-06-10,0.001122,0.015033
2024-06-11,0.010837,0.004388


#### 1. Criando a carteira

#### Cálculo da variância da carteira: com 2 ativos

$$ \sigma_{p}² = \sigma_{1}^{2} w_1^2 + \sigma_{2}^{2} w_2^2 + 2 \sigma_1 \sigma_2 w_1 w_2 \rho_{12}$$

In [4]:
import numpy as np
import pandas as pd
import plotly.express as px


# Dados de retorno diário das ações BBAS3 e PETR4
df = retorno


# Função para calcular o retorno e risco da carteira
def carteira_calculo(df, weights):
    carteira_list = []
    correlation = df.corr()
    rho_12 = correlation.iloc[0, 1]
    std_dev = df.std()
    sigma_1, sigma_2 = std_dev.iloc[0], std_dev.iloc[1]
    mu_1, mu_2 = df.mean().iloc[0], df.mean().iloc[1]

    for w_1 in weights:
        w_2 = 1 - w_1
        retorno = (mu_1 * w_1 + mu_2 * w_2) * 252
        risco = np.sqrt((sigma_1**2 * w_1**2 + sigma_2**2 * w_2**2 + 2 * rho_12 * w_1 * w_2 * sigma_1 * sigma_2) * 252)
        carteira_list.append({'retorno': retorno, 'risco': risco, 'pesos': w_1})
    
    carteira = pd.DataFrame(carteira_list)
    return carteira

# Pesos para a carteira
weights = np.linspace(start=0, stop=1, num=200)

# Calculando a carteira
carteira = carteira_calculo(df, weights)

carteira['retorno(%)'] = carteira['retorno'] * 100
carteira['risco(%)'] = carteira['risco'] * 100

# Plotando o gráfico do retorno vs risco da carteira
fig = px.scatter(
    carteira,
    x='risco(%)',
    y='retorno(%)',
    color='pesos',
    title='Retorno vs Risco da Carteira',
    labels={'risco(%)': 'Risco (%)', 'retorno(%)': 'Retorno (%)', 'color': 'Risco (%)'},
    hover_data={'pesos': True, 'risco(%)': ':.2f', 'retorno(%)': ':.2f'}
)

# Adicionando a legenda personalizada
fig.update_layout(
    legend_title_text='BBAS3'
)

fig.show()